## Treinamento do Modelo CNN

O código abaixo define e treina uma Rede Neural Convolucional (CNN) simples (uma arquitetura baseline) usando o gerador de dados do script_carregar_dados.py.

Ao final do treinamento, o modelo é salvo no formato .keras, que é o arquivo que será carregado e usado para avaliação no script_avaliacao.py.

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from script_carregar_dados import create_generators, IMAGE_SIZE, BATCH_SIZE
import os

# --- 1. CONFIGURAÇÕES E CARREGAMENTO DOS DADOS ---
DATASET_PATH = 'chars74k_dataset_final'
MODEL_NAME = 'char_recognition_cnn.keras'
INPUT_SHAPE = (*IMAGE_SIZE, 1) # (64, 64, 1) para imagens em escala de cinza

# 1.1. Geração dos Data Generators
print("Iniciando a criação dos Data Generators...")
train_gen, val_gen, test_gen = create_generators(DATASET_PATH)

# O número de classes é inferido pelo flow_from_directory
NUM_CLASSES = train_gen.num_classes
print(f"Número de classes inferidas: {NUM_CLASSES}")

# --- 2. DEFINIÇÃO DA ARQUITETURA DA CNN (Baseline) ---

model = Sequential([
    # Camada Convolucional 1
    Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
    MaxPooling2D((2, 2)),
    
    # Camada Convolucional 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Camada Convolucional 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flatten para alimentar a rede densa
    Flatten(),
    
    # Camada Densa 1
    Dense(512, activation='relu'),
    Dropout(0.5), # Regularização para evitar Overfitting
    
    # Camada de Saída
    Dense(NUM_CLASSES, activation='softmax') # Softmax para classificação multi-classe
])

# --- 3. COMPILAÇÃO DO MODELO ---

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', # Perda padrão para classificação multi-classe
    metrics=['accuracy']
)

model.summary()

# --- 4. CALLBACKS PARA TREINAMENTO ---

# 4.1. Model Checkpoint: Salva o melhor modelo (baseado na acurácia da validação)
checkpoint = ModelCheckpoint(
    MODEL_NAME, 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max', 
    verbose=1
)

# 4.2. Early Stopping: Para o treinamento se a métrica não melhorar após algumas épocas
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, # Espera por 5 épocas sem melhoria
    restore_best_weights=True
)

callbacks_list = [checkpoint, early_stopping]

# --- 5. TREINAMENTO DO MODELO ---

print("\nIniciando o treinamento...")

# O model.fit_generator (antigo) foi substituído por model.fit com generators
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // BATCH_SIZE,
    epochs=50, # Defina um número alto e confie no Early Stopping
    validation_data=val_gen,
    validation_steps=val_gen.samples // BATCH_SIZE,
    callbacks=callbacks_list,
    verbose=1
)

print(f"\n✅ Treinamento concluído. O melhor modelo foi salvo como '{MODEL_NAME}'.")

Iniciando a criação dos Data Generators...
Found 51804 images belonging to 62 classes.
Found 7367 images belonging to 62 classes.
Found 7371 images belonging to 62 classes.
Número de classes inferidas: 62


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 62)             │        31,806 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,484,286 (9.48 MB)

 Trainable params: 2,484,286 (9.48 MB)

 Non-trainable params: 0 (0.00 B)


Iniciando o treinamento...
Epoch 1/50
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.4135 - loss: 2.2487
Epoch 1: val_accuracy improved from None to 0.80041, saving model to char_recognition_cnn.keras
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 388s 238ms/step - accuracy: 0.5870 - loss: 1.4812 - val_accuracy: 0.8004 - val_loss: 0.6768
Epoch 2/50
   1/1618 ━━━━━━━━━━━━━━━━━━━━ 6:34 244ms/step - accuracy: 0.7188 - loss: 0.9911
Epoch 2: val_accuracy did not improve from 0.80041
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.7188 - loss: 0.9911 - val_accuracy: 0.7984 - val_loss: 0.6770
Epoch 3/50
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.7424 - loss: 0.8259
Epoch 3: val_accuracy improved from 0.80041 to 0.82255, saving model to char_recognition_cnn.keras
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 350s 216ms/step - accuracy: 0.7510 - loss: 0.8017 - val_accuracy: 0.8226 - val_loss: 0.5812
Epoch 4/50
   1/1618 ━━━━━━━━━━━━━━━━━━━━ 9:28 351ms/step - accuracy: 0.8125 - loss: 0.

### Significado de `conv2d_6`

O `_6` após `conv2d` é um **contador de instância** atribuído automaticamente pelo *framework* (Keras ou TensorFlow) para identificar unicamente a camada.

* **`conv2d`**: Indica o tipo de camada (Camada Convolucional 2D).
* **`_6`**: Significa que esta é a **sexta camada convolucional 2D** que o Keras encontrou e nomeou desde o início do seu modelo ou *notebook*.

Se não especificar um nome manualmente ao criar a camada (ex: `Conv2D(..., name='minha_conv')`), o Keras usa essa convenção de nomenclatura para evitar conflitos.

---

### Parâmetros de Saída (`Output Shape`)

A tupla **`(None, 62, 62, 32)`** descreve a forma do **tensor de dados** que esta camada irá produzir e passar para a próxima camada.

| Posição | Valor | Significado |
| :---: | :---: | :--- |
| 1º | **`None`** | **Tamanho do Lote (*Batch Size*):** Significa que o modelo pode aceitar qualquer tamanho de lote (número de imagens processadas simultaneamente). Este valor só será definido durante o treinamento (ex: 32). |
| 2º | **`62`** | **Altura:** A altura do mapa de características de saída (em pixels). |
| 3º | **`62`** | **Largura:** A largura do mapa de características de saída (em pixels). |
| 4º | **`32`** | **Profundidade/Canais:** O número de **filtros (kernels)** usados nesta camada $\text{CONV}$. |

**Conclusão:** Esta camada pega uma entrada (provavelmente $64 \times 64$) e, após aplicar a convolução, produz 32 mapas de características, cada um com dimensões de $62 \times 62$ pixels.

---

### `Param #` (320)

O **`Param #`** (Número de Parâmetros) representa o número total de **pesos** e **vieses (*biases*)** que o modelo precisa **aprender** ou ajustar durante o treinamento nesta camada específica.

O cálculo para a camada $\text{CONV}$ segue a fórmula:

$$\text{Params} = ((\text{Tamanho do Filtro} \times \text{Tamanho do Filtro} \times \text{Canais de Entrada}) + 1) \times \text{Canais de Saída}$$

O cálculo para $\text{320}$ sugere:

1.  **Canais de Saída:** $32$ (dado no `Output Shape`).
2.  **Canais de Entrada:** $1$ (Se a camada anterior for a imagem de entrada em escala de cinza, $64 \times 64 \times 1$).
3.  **Tamanho do Filtro:** $3 \times 3$ (Valor padrão se não especificado).

**Cálculo:**
$$\text{Params} = ((3 \times 3 \times 1) + 1) \times 32$$
$$\text{Params} = (9 + 1) \times 32$$
$$\text{Params} = 10 \times 32 = 320$$

**Conclusão:** O valor **320** representa o número de **pesos ajustáveis** nesta camada, que são os pesos dos 32 filtros $3 \times 3$ e seus respectivos vieses.